In [1]:
import copy
import csv
import os
import warnings
from argparse import ArgumentParser

import torch
import tqdm
import yaml
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
import glob
import json
import os
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda:0


In [ ]:
# 파라미터 및 데이터 경로 설정
with open('utils/Binary_args.yaml', errors='ignore') as f:
    params = yaml.safe_load(f)

print("모델 파라미터:")
print(f"클래스 이름: {params['names']}")
print(f"클래스 수: {len(params['names'])}")

label_dir = '../../data/IGNITE/annotations/pdl1/binary_individual/'
image_dir = '../../data/IGNITE/images/pdl1/nuclei/'

label_files = sorted(glob.glob(os.path.join(label_dir, '*.json')))
filenames = []
labels = []

print(f"첫 번째 데이터셋에서 찾은 라벨 파일: {len(label_files)}개")

for label_file in label_files:
    with open(label_file) as f:
        data1 = json.load(f)
    img_path = os.path.join(image_dir, data1['image']['file_name'])
    if os.path.exists(img_path):
        filenames.append(img_path)
        temp_labels = []
        for i in range(len(data1['annotations'])):
            # 모든 nucleus를 클래스 1로 통일
            temp_labels.append([1, int(data1['annotations'][i]['bbox'][0]),
                         int(data1['annotations'][i]['bbox'][1]),int(data1['annotations'][i]['bbox'][2]),int(data1['annotations'][i]['bbox'][3])])
            
        labels.append(temp_labels)
        
label_dir = '../../data/IGNITE/annotations/pdl1/individual/'
image_dir = '../../data/IGNITE/images/pdl1/pdl1/'

label_files = sorted(glob.glob(os.path.join(label_dir, '*.json')))
print(f"두 번째 데이터셋에서 찾은 라벨 파일: {len(label_files)}개")

for label_file in label_files:
    with open(label_file) as f:
        data1 = json.load(f)
    img_path = os.path.join(image_dir, data1['image']['file_name'])
    if os.path.exists(img_path):
        filenames.append(img_path)
        temp_labels = []
        for i in range(len(data1['annotations'])):
            # 모든 nucleus를 클래스 1로 통일 (이미 1로 설정되어 있음)
            temp_labels.append([1, int(data1['annotations'][i]['bbox'][0]),
                         int(data1['annotations'][i]['bbox'][1]),int(data1['annotations'][i]['bbox'][2]),int(data1['annotations'][i]['bbox'][3])])
        labels.append(temp_labels)

print(f"\n총 데이터:")
print(f"이미지 파일: {len(filenames)}개")
print(f"라벨 세트: {len(labels)}개")

# 라벨 분포 확인
total_labels = 0
class_counts = {}
for label_set in labels:
    total_labels += len(label_set)
    for label in label_set:
        class_id = label[0]
        if class_id not in class_counts:
            class_counts[class_id] = 0
        class_counts[class_id] += 1

print(f"\n라벨 분포:")
print(f"총 라벨 수: {total_labels}")
for class_id, count in sorted(class_counts.items()):
    print(f"클래스 {class_id}: {count}개")

# 빈 라벨 세트 확인
empty_label_count = sum(1 for label_set in labels if len(label_set) == 0)
print(f"빈 라벨 세트: {empty_label_count}개")

if empty_label_count > 0:
    print("Warning: 빈 라벨 세트가 있습니다. 이는 loss가 0이 되는 원인이 될 수 있습니다.")

In [ ]:
class custom_dataset(data.Dataset):
    def __init__(self, filenames, input_size, params, augment, labels=None, image_infos=None):
        self.params = params
        self.mosaic = augment
        self.augment = augment
        self.input_size = input_size
        if labels is not None:
            self.labels = labels
            self.filenames = filenames
            self.n = len(self.filenames)
            self.image_infos = image_infos if image_infos is not None else [None]*len(filenames)
        else:
            loaded = self.load_label(filenames)
            self.labels = list(loaded.values())
            self.filenames = list(loaded.keys())
            self.n = len(self.filenames)
            self.image_infos = [None]*self.n
        self.indices = range(self.n)
        self.albumentations = Albumentations()
    def __len__(self):
        return self.n
    def __getitem__(self, index):
        index = self.indices[index]
        temp_label = copy.deepcopy(self.labels[index])
        
        image,crop_index=self.load_image(index)
        
        crop_x, crop_y = crop_index
        label=[]
        #bbox 좌표를 YOLO 형식으로 변환: x_center, y_center, width, height (모두 0~1 정규화)
        for i in range(len(temp_label)):
            x = temp_label[i][1]  # bbox x
            y = temp_label[i][2]  # bbox y  
            w = temp_label[i][3]  # bbox width
            h = temp_label[i][4]  # bbox height
            
            # 바운딩 박스가 크롭된 영역 내에 있는지 확인 (더 관대한 조건)
            # 바운딩 박스의 중심점이 크롭 영역 내에 있으면 포함
            center_x = x + w/2
            center_y = y + h/2
            
            if (center_x >= crop_x and center_y >= crop_y and 
                center_x <= crop_x + self.input_size and center_y <= crop_y + self.input_size):
                
                # YOLO 형식으로 변환: (x_center, y_center, width, height) - 모두 0~1 사이 정규화
                norm_x_center = (center_x - crop_x) / self.input_size
                norm_y_center = (center_y - crop_y) / self.input_size
                norm_width = w / self.input_size
                norm_height = h / self.input_size
                
                # 정규화된 값들이 유효한 범위에 있는지 확인
                if (0 <= norm_x_center <= 1 and 0 <= norm_y_center <= 1 and 
                    norm_width > 0 and norm_height > 0):
                    
                    # YOLO 형식: [class, x_center, y_center, width, height]
                    converted_label = [temp_label[i][0], norm_x_center, norm_y_center, norm_width, norm_height]
                    label.append(converted_label)

        # 디버깅 정보 추가
        if len(label) == 0 and len(temp_label) > 0:
            print(f"Warning: 파일 {self.filenames[index]}에서 크롭 후 라벨이 없습니다. 원본 라벨 수: {len(temp_label)}")
            print(f"크롭 정보: crop_x={crop_x}, crop_y={crop_y}, input_size={self.input_size}")

        cls=[]
        box=[]
        for i in range(len(label)):
            cls.append(label[i][0])
            box.append(label[i][1:5])
        
        # 클래스 인덱스를 0부터 시작하도록 변경 (1 -> 0)
        cls=np.array(cls, dtype=np.float32)
        if len(cls) > 0:
            cls = cls - 1  # 1 -> 0 변환 (단일 클래스 detection)
            cls = np.clip(cls, 0, len(self.params['names'])-1)  # 유효 범위로 클리핑
        
        box=np.array(box, dtype=np.float32)
        nl = len(box)
        
        if self.augment and nl > 0:  # 라벨이 있을 때만 augmentation 적용
            # Flip up-down
            if random.random() < self.params['flip_ud']:
                image = np.flipud(image).copy()
                if nl:
                    box[:, 1] = 1 - box[:, 1]  # y_center 반전
            # Flip left-right
            if random.random() < self.params['flip_lr']:
                image = np.fliplr(image).copy()
                if nl:
                    box[:, 0] = 1 - box[:, 0]  # x_center 반전

        image = image.transpose((2, 0, 1))
        
        # 빈 텐서 대신 적절한 크기의 인덱스 텐서 반환
        return (torch.from_numpy(image).float(), 
                torch.from_numpy(cls).long(), 
                torch.from_numpy(box).float(), 
                torch.full((nl,), index, dtype=torch.long))

    def load_image(self, i):
        image = cv2.imread(self.filenames[i])
        if image is None:
            raise ValueError(f"이미지를 불러올 수 없습니다: {self.filenames[i]}")
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        h, w = image.shape[:2]
        r = self.input_size / min(h, w)
        
        # 이미지가 input_size보다 큰 경우 (r < 1) -> 랜덤 크롭
        if r < 1:
            # 안전하게 크롭 범위 계산
            max_h = max(0, h - self.input_size)
            max_w = max(0, w - self.input_size)
            h1 = random.randint(0, max_h) if max_h > 0 else 0
            w1 = random.randint(0, max_w) if max_w > 0 else 0
            image = image[h1:h1 + self.input_size, w1:w1 + self.input_size]
        else:
            # 이미지가 input_size보다 작거나 같은 경우 (r >= 1) -> 패딩
            h1 = 0
            w1 = 0
            pad_image = np.ones((self.input_size, self.input_size, 3), dtype=np.uint8)*114  # 회색 패딩
            pad_image[:min(h,self.input_size), :min(w,self.input_size), :] = image[:min(h,self.input_size), :min(w,self.input_size), :]
            image = pad_image
        return image, (w1, h1)  # x, y 순서로 반환


    
    
class Albumentations:
    def __init__(self):
        self.transform = None
        try:
            import albumentations

            transforms = [albumentations.Blur(p=0.01),
                          albumentations.CLAHE(p=0.01),
                          albumentations.ToGray(p=0.01),
                          albumentations.MedianBlur(p=0.01)]
            self.transform = albumentations.Compose(transforms,
                                                    albumentations.BboxParams('yolo', ['class_labels']))

        except ImportError:  # package not installed, skip
            pass

    def __call__(self, image, box, cls):
        if self.transform:
            x = self.transform(image=image,
                               bboxes=box,
                               class_labels=cls)
            image = x['image']
            box = np.array(x['bboxes'])
            cls = np.array(x['class_labels'])
        return image, box, cls

split=[0.9, 0.1]
train_dataset=custom_dataset(filenames[:int(len(filenames)*split[0])],512, params, augment=True, labels=labels[:int(len(filenames)*split[0])])
val_dataset = custom_dataset(filenames[int(len(filenames)*split[0]):],512, params, augment=False, labels=labels[int(len(filenames)*split[0]):])

# 데이터셋 검증
print(f"전체 데이터: {len(filenames)}")
print(f"훈련 데이터: {len(train_dataset)}")
print(f"검증 데이터: {len(val_dataset)}")

# 샘플 데이터 확인
sample_idx = 0
sample_image, sample_cls, sample_box, sample_indices = train_dataset[sample_idx]
print(f"\n샘플 데이터 확인:")
print(f"이미지 크기: {sample_image.shape}")
print(f"클래스 수: {len(sample_cls)}")
print(f"바운딩 박스 수: {len(sample_box)}")
if len(sample_cls) > 0:
    print(f"클래스 범위: {sample_cls.min():.0f} ~ {sample_cls.max():.0f}")
if len(sample_box) > 0:
    print(f"바운딩 박스 좌표 범위: x_center={sample_box[:, 0].min():.3f}~{sample_box[:, 0].max():.3f}, y_center={sample_box[:, 1].min():.3f}~{sample_box[:, 1].max():.3f}")
    print(f"바운딩 박스 크기 범위: width={sample_box[:, 2].min():.3f}~{sample_box[:, 2].max():.3f}, height={sample_box[:, 3].min():.3f}~{sample_box[:, 3].max():.3f}")

/usr/local/lib/python3.12/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:251: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [4]:
def collate_fn1(batch):
    samples, cls, box, indices = zip(*batch)

    cls = torch.cat(cls, dim=0)
    box = torch.cat(box, dim=0)

    new_indices = list(indices)
    for i in range(len(indices)):
        new_indices[i] += i
    indices = torch.cat(new_indices, dim=0)

    targets = {'cls': cls,
                'box': box,
                'idx': indices}
    return torch.stack(samples, dim=0), targets


# 모델 및 파라미터 준비
model = nn.yolo_v11_m(len(params['names'])).to(device)
optimizer = torch.optim.SGD(util.set_params(model, params['weight_decay']),
                            params['min_lr'], params['momentum'], nesterov=True)
criterion = util.ComputeLoss(model, params)

# 데이터셋 및 데이터로드 (안전한 함수 사용)
batch_size = 4
# 안전하게 데이터로더 생성하는 함수
def create_safe_loader(dataset, batch_size, is_train=True):
    """
    배치 크기에 맞게 데이터셋을 조정하여 안전하게 데이터로더를 생성하는 함수
    """
    dataset_size = len(dataset)
    
    # 배치 크기가 데이터셋 크기보다 큰 경우 배치 크기 조정
    if dataset_size < batch_size:
        print(f"경고: 데이터셋 크기({dataset_size})가 배치 크기({batch_size})보다 작습니다. 배치 크기를 {dataset_size}로 조정합니다.")
        actual_batch_size = max(1, dataset_size)
    else:
        actual_batch_size = batch_size
    
    # 데이터셋이 배치 크기로 나누어 떨어지는지 확인
    if dataset_size % actual_batch_size != 0:
        print(f"참고: 데이터셋 크기({dataset_size})가 배치 크기({actual_batch_size})로 나누어 떨어지지 않습니다.")
        print(f"마지막 배치는 {dataset_size % actual_batch_size}개의 샘플을 포함합니다.")
    
    # 데이터로더 생성
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=actual_batch_size, 
        shuffle=is_train,
        num_workers=4,
        collate_fn=collate_fn1,
        drop_last=(not is_train)  # 훈련 시에는 마지막 배치 유지, 검증 시에는 마지막 배치 제외
    )
    
    return loader, actual_batch_size
# 안전하게 데이터로더 생성
loader, train_batch_size = create_safe_loader(train_dataset, batch_size, is_train=True)
val_loader, val_batch_size = create_safe_loader(val_dataset, 1, is_train=False)

print(f"최종 훈련 배치 크기: {train_batch_size}")
print(f"최종 검증 배치 크기: {val_batch_size}")



참고: 데이터셋 크기(431)가 배치 크기(4)로 나누어 떨어지지 않습니다.
마지막 배치는 3개의 샘플을 포함합니다.
최종 훈련 배치 크기: 4
최종 검증 배치 크기: 1


In [ ]:
from utils.valid import compute_validation_metrics, compute_validation_metrics_with_kappa, get_kappa_interpretation
from utils.valid import visualize_ground_truth_and_prediction_separately
from utils.valid import plot_training_progress


# main.py의 train 함수를 참조한 개선된 학습 루프
train_losses = []
val_maps = []
val_precisions = []
val_recalls = []
val_map50s = []
val_kappas = []  # Cohen's Kappa 추가
epochs = 10000

# 체크포인트 저장을 위한 디렉토리 생성
save_dir = '../../model/binary_yolov11/'
os.makedirs(save_dir, exist_ok=True)
#체크포인트 불러오기 
# checkpoint_path = os.path.join(save_dir, 'best_model.pt')
# if os.path.exists(checkpoint_path):
#     checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
#     model.load_state_dict(checkpoint['model_state_dict'])
    
# main.py 스타일의 설정들
best_map = 0
accumulate = max(round(64 / batch_size), 1)  # gradient accumulation steps
amp_scale = torch.amp.GradScaler()  # mixed precision scaler

print(f"Gradient accumulation steps: {accumulate}")

for epoch in range(epochs):
    # 훈련
    model.train()
    
    # main.py 스타일의 평균 손실 추적
    avg_box_loss = util.AverageMeter()
    avg_cls_loss = util.AverageMeter()
    avg_dfl_loss = util.AverageMeter()
    
    train_pbar = tqdm.tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {epoch+1}/{epochs} Training')
    
    optimizer.zero_grad()
    
    for i, (images, targets) in train_pbar:
        step = i + len(loader) * epoch
        images = images.to(device).float() / 255
        
        # Forward pass with mixed precision
        with torch.amp.autocast('cuda'):
            outputs = model(images)
            loss_box, loss_cls, loss_dfl = criterion(outputs, targets)
        
        # 평균 손실 업데이트
        avg_box_loss.update(loss_box.item(), images.size(0))
        avg_cls_loss.update(loss_cls.item(), images.size(0))
        avg_dfl_loss.update(loss_dfl.item(), images.size(0))
        
        # Loss scaling for gradient accumulation
        loss_box *= batch_size  # loss scaled by batch_size
        loss_cls *= batch_size  # loss scaled by batch_size  
        loss_dfl *= batch_size  # loss scaled by batch_size
        
        total_loss = loss_box + loss_cls + loss_dfl
        
        # Backward pass with gradient scaling
        amp_scale.scale(total_loss).backward()
        
        # Gradient accumulation 및 optimization
        if step % accumulate == 0:
            # Gradient clipping 및 optimization
            amp_scale.step(optimizer)
            amp_scale.update()
            optimizer.zero_grad()
        
        # GPU 메모리 동기화
        torch.cuda.synchronize()
        
        # 진행률 표시 업데이트 (main.py 스타일)
        memory = f'{torch.cuda.memory_reserved() / 1E9:.4g}G'
        s = f'Memory: {memory} | Box: {avg_box_loss.avg:.3f} | Cls: {avg_cls_loss.avg:.3f} | DFL: {avg_dfl_loss.avg:.3f}'
        train_pbar.set_description(f'Epoch {epoch+1}/{epochs} | {s}')
    
    # 에폭 평균 손실 계산
    avg_train_loss = avg_box_loss.avg + avg_cls_loss.avg + avg_dfl_loss.avg
    train_losses.append(avg_train_loss)
    
    # 검증 (Cohen's Kappa 포함)
    precision, recall, map50, mean_ap, kappa = compute_validation_metrics_with_kappa(
        model, val_loader, device, params
    )
    val_maps.append(mean_ap)
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_map50s.append(map50)
    val_kappas.append(kappa)
    
    # F1-score 계산
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # 결과 출력 (Cohen's Kappa 포함)
    print(f"\nEpoch {epoch+1}/{epochs} Results:")
    print(f"  Train Loss - Box: {avg_box_loss.avg:.4f}, Cls: {avg_cls_loss.avg:.4f}, DFL: {avg_dfl_loss.avg:.4f}, Total: {avg_train_loss:.4f}")
    print(f"  Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}")
    print(f"  mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {mean_ap:.4f}")
    print(f"  Cohen's Kappa: {kappa:.4f} ({get_kappa_interpretation(kappa)})")
    print("-" * 80)
    
    # 베스트 모델 저장 (mAP 기준)
    if mean_ap > best_map:
        best_map = mean_ap
        save_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'amp_scale_state_dict': amp_scale.state_dict(),
            'train_loss': avg_train_loss,
            'box_loss': avg_box_loss.avg,
            'cls_loss': avg_cls_loss.avg,
            'dfl_loss': avg_dfl_loss.avg,
            'map': mean_ap,
            'map50': map50,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'kappa': kappa,
            'params': params
        }
        torch.save(save_checkpoint, os.path.join(save_dir, 'best_model.pt'))
        print(f"🎉 새로운 베스트 모델 저장! mAP: {mean_ap:.4f}, Kappa: {kappa:.4f}")
    
    # 최신 모델도 저장 (main.py 스타일)
    last_checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'amp_scale_state_dict': amp_scale.state_dict(),
        'train_loss': avg_train_loss,
        'box_loss': avg_box_loss.avg,
        'cls_loss': avg_cls_loss.avg,
        'dfl_loss': avg_dfl_loss.avg,
        'map': mean_ap,
        'map50': map50,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'kappa': kappa,
        'params': params
    }
    torch.save(last_checkpoint, os.path.join(save_dir, 'last_model.pt'))
    
    # 100 에폭마다 학습 진행 그래프 생성 및 저장
    if (epoch + 1) % 100 == 0:
        try:
            print(f"\n📊 Epoch {epoch+1} - 학습 진행 상황 그래프 생성 중...")
            plot_training_progress(train_losses, val_maps, val_precisions, val_recalls, val_map50s, epoch+1, save_dir)
        except Exception as e:
            print(f"그래프 생성 중 오류: {e}")
    
    # 개선된 검증 이미지 시각화 (매 10 에폭마다) - 실제 라벨과 예측 라벨을 별도 figure로 표시
    if (epoch + 1) % 10 == 0:
        try:
            # 여러 샘플에 대해 시각화
            num_samples = 1 # 샘플 수를 1개
            for sample_idx in range(num_samples):
                print(f"\n📊 Epoch {epoch+1} - 검증 샘플 {sample_idx+1}/{num_samples}:")
                print("=" * 60)
                
                # 실제 라벨과 예측 라벨을 별도 figure로 표시
                sample_idx = random.randint(0, len(val_dataset)-1)
                visualize_ground_truth_and_prediction_separately(
                    model, val_dataset, idx=sample_idx, 
                    epoch=epoch+1, save_dir=save_dir
                )
                
        except Exception as e:
            print(f"시각화 중 오류: {e}")

print("🎯 학습 완료!")
print(f"최종 베스트 mAP: {best_map:.4f}")
print(f"모델 저장 위치: {save_dir}")
print(f"베스트 모델: {os.path.join(save_dir, 'best_model.pt')}")
print(f"최신 모델: {os.path.join(save_dir, 'last_model.pt')}")

# 최종 성능 요약
if val_kappas:
    final_kappa = val_kappas[-1]
    final_map = val_maps[-1]
    final_precision = val_precisions[-1]
    final_recall = val_recalls[-1]
    final_f1 = 2 * (final_precision * final_recall) / (final_precision + final_recall) if (final_precision + final_recall) > 0 else 0
    
    print(f"\n📊 최종 성능 요약:")
    print(f"  mAP@0.5:0.95: {final_map:.4f}")
    print(f"  Cohen's Kappa: {final_kappa:.4f} ({get_kappa_interpretation(final_kappa)})")
    print(f"  F1-score: {final_f1:.4f}")
    print(f"  Precision: {final_precision:.4f}")
    print(f"  Recall: {final_recall:.4f}")

Gradient accumulation steps: 16


Epoch 1/10000 Training:   0%|          | 0/108 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3611.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Epoch 1/10000 | Memory: 2.926G | Box: 3.008 | Cls: 8.548 | DFL: 1.232: 100%|██████████| 108/108 [00:13<00:00,  7.96it/s]



Epoch 1/10000 Results:
  Train Loss - Box: 3.0081, Cls: 8.5483, DFL: 1.2323, Total: 12.7887
  Validation - Precision: 0.1011, Recall: 0.0037, F1-score: 0.0072
  mAP@0.5: 0.0510, mAP@0.5:0.95: 0.0207
  Cohen's Kappa: 0.0372 (Slight)
--------------------------------------------------------------------------------
🎉 새로운 베스트 모델 저장! mAP: 0.0207, Kappa: 0.0372


Epoch 2/10000 | Memory: 2.926G | Box: 3.025 | Cls: 9.864 | DFL: 1.232: 100%|██████████| 108/108 [00:11<00:00,  9.21it/s]



Epoch 2/10000 Results:
  Train Loss - Box: 3.0253, Cls: 9.8636, DFL: 1.2316, Total: 14.1205
  Validation - Precision: 0.0799, Recall: 0.0029, F1-score: 0.0056
  mAP@0.5: 0.0404, mAP@0.5:0.95: 0.0153
  Cohen's Kappa: 0.0330 (Slight)
--------------------------------------------------------------------------------


...


Epoch 27/10000 Results:
  Train Loss - Box: 0.0000, Cls: 0.0000, DFL: 0.0000, Total: 0.0000
  Validation - Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
  mAP@0.5: 0.0000, mAP@0.5:0.95: 0.0000
  Cohen's Kappa: 0.0000 (Slight)
--------------------------------------------------------------------------------


Epoch 28/10000 | Memory: 3.592G | Box: 0.000 | Cls: 0.000 | DFL: 0.000:  56%|█████▋    | 61/108 [00:06<00:05,  8.75it/s]


KeyboardInterrupt: 

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>